In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow.keras import models , layers
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil

import pandas as pd

In [ ]:
#Set the constaant values
batch_size = 32
image_size = 256
channels = 3
epochs = 10
dataset = tf.keras.utils.image_dataset_from_directory(directory="/kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data/train",batch_size = batch_size , image_size = (image_size,image_size))

In [ ]:
class_names = dataset.class_names

In [ ]:
def dataset_split(ds,train_split=0.7,val_split=0.15,test_split=0.15,shuffle_size=10000):
    ds_size = len(ds) #df_size should be equal to the size of data
    if shuffle_size:
        ds=ds.shuffle(shuffle_size,seed=42) #shuffle the data randomly
        train_size = int(train_split*ds_size)
        val_size = int(val_split*ds_size)
        train_ds = ds.take(train_size)
        val_ds = ds.skip(train_size).take(val_size)
        test_ds = ds.skip(train_size).skip(val_size)
        print(f"Train size : {train_ds}")
        print(f"Test size : {test_ds}")
        print(f"Validation size : {val_ds}")
        return train_ds,val_ds,test_ds

In [ ]:
train_ds , validation_ds ,test_ds=dataset_split(dataset)

In [ ]:
#plot the data
plt.figure(figsize=(15,15))
for image_batch,label_batch in dataset.take(1):
    for i in range(12):
        ax=plt.subplot(3,4,i+1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.axis("off")
        plt.title(class_names[label_batch[i]])

In [ ]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=validation_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(image_size,image_size), #convert each image to 255,255
    layers.experimental.preprocessing.Rescaling(1./255) # Normalize each image to 0-1
])

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
])
input_size = (batch_size,image_size,image_size,channels) #(32,256,256,3)

In [ ]:
out = len(class_names)

In [ ]:
#model
model = tf.keras.models.Sequential([
    data_augmentation,
    layers.Conv2D(16,(3,3),activation="relu",input_shape=input_size),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(32,(3,3),activation="relu"),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(196,activation="relu"),
    layers.Dense(1,activation="softmax")
])
model.build(input_shape=input_size)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss=tf.keras.losses.categorical_crossentropy,metrics=["accuracy"])

In [ ]:
history = model.fit(train_ds,epochs=epochs,batch_size=batch_size,verbose=1,validation_data=val_ds)

In [ ]:
#Matplotlib
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
val_loss=history.history['val_loss']
loss=history.history['loss']

plt.figure(figsize=(8,4))
plt.plot(range(epochs),acc,label="Training Acc")
plt.plot(range(epochs),val_acc,label="Val Acc")
plt.legend(loc="lower right")
plt.title('Training and Validation Accuracy')

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(range(epochs),loss,label="Training loss")
plt.plot(range(epochs),val_loss,label="Val loss")
plt.legend(loc="lower left")
plt.title('Training and Validation loss')